In [ ]:
!pip install pvporcupine
!pip install pvrecorder
!pip install SpeechRecognition
!pip install openai
!pip install gTTS

In [1]:
import pvporcupine
from pvrecorder import PvRecorder
import speech_recognition as sr
import openai
import gtts
import IPython
import time
from IPython.display import display

In [3]:
porcupine = pvporcupine.create(access_key="your_pvporcupine_api_key_here", keywords=["alexa"])
openai.api_key = "your_openai_api_key_here"
recoder = PvRecorder(device_index=-1, frame_length=porcupine.frame_length)

try:
    recoder.start()

    while True:
        keyword_index = porcupine.process(recoder.read())
        if keyword_index >= 0:
            print("Detected keyword")
            display(IPython.display.Audio("hi.mp3", autoplay=True))
            time.sleep(0.5)
            
            r = sr.Recognizer()
            with sr.Microphone() as source:
                display(IPython.display.Audio("preorder.mp3", autoplay=True))
                print("Place your order :")
                time.sleep(1)
                audio = r.listen(source)
                try:
                    text = r.recognize_google(audio)
                    print("You said : {}".format(text))
                    response = openai.ChatCompletion.create(
                        model = "gpt-3.5-turbo",
                        messages=[
                            {"role":"user", "content":"Here is the menu for my restaurant (Dev's Restaurant) along with the prices: Main Dishes: 1. Cheese burger - $8. 2. Large Veggie Pizza - $12. 3. Small Veggie Pizza - $8. Side Dishes: 1. Fries - $3. 2. Hashbrowns - $5. Dessert: 1. Vanilla ice cream - $3. 2. Chocolate cake - $5. My customers will place an order. I want you to take their order and reply with a receipt."},
                            {"role":"user", "content":text}
                        ]
                    )
                    order_text = response["choices"][0]["message"]["content"]
                    order_text = order_text.replace("-", "")
                    print(order_text)
                    order = gtts.gTTS(text=order_text)
                    order.save("order.mp3")
                    display(IPython.display.Audio("order.mp3", autoplay=True))
                except:
                    print("Sorry could not recognize what you said")

except KeyboardInterrupt:
    recoder.stop()
finally:
    porcupine.delete()
    recoder.delete()

Detected keyword


Place your order :
You said : my name is Dave can I please get a cheese burger with french fries and vanilla ice cream
Sure, Dave! Thank you for choosing Dev's Restaurant. Here's your order and the receipt:

Order:
 Cheese burger  $8
 Fries  $3
 Vanilla ice cream  $3

Receipt:
Item                  Price

Cheese burger         $8.00
Fries                  $3.00
Vanilla ice cream      $3.00

Total:               $14.00

Thank you for your order, Dave! Your total is $14.00. Please let us know if there's anything else we can assist you with.
